# Connecting to free language learing model APIs

## 1. Co:here

In [2]:
# import libraries
import pandas as pd
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [3]:
# get the data
df = pd.read_excel("../data/Example_data.xlsx")

In [4]:
# look at data
df.head(1)

Domain  \
0  rassegnastampa.news   

                                                                                                                       Title  \
0  Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for   

                                                                                                                                                                                                                                                   Description  \
0  …often trigger a protest vote that can upset…that ministers who breach the Ministerial Code…plane up the M1 in order to…serious questions about abuse of taxpayers' money which is a clear breach of the code.…should not permit abuse of taxpayers' money…   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [7]:
df.shape

(10, 9)

In [8]:
# df.index()

In [9]:
# create a sample dataset
num_examples = 5
df_sample = df.sample(num_examples)

In [10]:
# split into training and testing sets
# we set the 'Body' column as the prompt and the 'Analyst_Average_Score' col as the target variable
sentences_train, sentences_test, labels_train, labels_test = train_test_split(list(df_sample['Body']), 
                                                                list(df_sample['Analyst_Average_Score']), 
                                                                test_size=0.25, 
                                                                random_state=0)

### Connecting to the Co:here API

In [11]:
# connecting to the api
import os
api_key = os.environ['COHERE_APIKEY']
# api_key = 

# creating and retrieving a cohere API key from os.cohere.ai
co = cohere.Client(api_key)

### a. Using the embed endpoint: Getting the embeddings of the item content

In [12]:
# embed the training set
embeddings_train = co.embed(texts=sentences_train,
                             model="large",
                             truncate="LEFT").embeddings
# embed the testing set
embeddings_test = co.embed(texts=sentences_test,
                             model="large",
                             truncate="LEFT").embeddings

We have created 2 sets of embeddings:
<li> embeddings_train - contains embeddings of the training sentences
<li> embeddings_test - contains embeddings of the testing sentences

In [13]:
# to see what an embedding looks like
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Review text: Global and Regional Beta-Carotene Market Research 2020 Report | Growth Forecast 2025 key players! – DSM – BASF – Allied Biotech – Chr Hansen – LYCORED Beta-Carotene Market Global Beta-Carotene market provides an in-depth insight into the global market covering all its important chapters which are must read for key industry stakeholders. Report coverage include historical market performance, impact of COVID-19, post-covid scenario of market, forecast for six years, market dynamics including growth drivers, challenges, market opportunities, Porter’s five forces analysis, value chain analysis. It offers a detailed analysis for establishing value chain and market product across other countries. It also presents detailed insights of market segments, their future growth potential and analysis of countries. In addition to the above, the information is based on the partners that are highly competitive, key players as well as their market revenue in the forecast years of 2020 to 20

### b. Using the generate endpoint: Creating a prompt that extracts movie name given a prompt
Reference: this cohere notebook: https://github.com/cohere-ai/notebooks/blob/main/notebooks/Entity_Extrcation_with_Generative_Language_Models.ipynb

In [14]:
# loading data
movie_examples = [
("Deadpool 2", "Deadpool 2 | Official HD Deadpool's \"Wet on Wet\" Teaser | 2018"),
("none", "Jordan Peele Just Became the First Black Writer-Director With a $100M Movie Debut"),
("Joker", "Joker Officially Rated “R”"),
("Free Guy", "Ryan Reynolds’ 'Free Guy' Receives July 3, 2020 Release Date - About a bank teller stuck in his routine that discovers he’s an NPC character in brutal open world game."),
("none", "James Cameron congratulates Kevin Feige and Marvel!"),
("Guardians of the Galaxy", "The Cast of Guardians of the Galaxy release statement on James Gunn"),
]

In [15]:
# creating a prompt to execute the required code
class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
      extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor([e[1] for e in movie_examples], 
                                       [e[0] for e in movie_examples], [],
                                       "", 
                                       "extract the movie title from the post:")


In [16]:
# displaying what generated prompt looks like
print(cohereMovieExtractor.make_prompt('<input text here>'))

Deadpool 2 | Official HD Deadpool's "Wet on Wet" Teaser | 2018
extract the movie title from the post:Deadpool 2
---
Jordan Peele Just Became the First Black Writer-Director With a $100M Movie Debut
extract the movie title from the post:none
---
Joker Officially Rated “R”
extract the movie title from the post:Joker
---
Ryan Reynolds’ 'Free Guy' Receives July 3, 2020 Release Date - About a bank teller stuck in his routine that discovers he’s an NPC character in brutal open world game.
extract the movie title from the post:Free Guy
---
James Cameron congratulates Kevin Feige and Marvel!
extract the movie title from the post:none
---
The Cast of Guardians of the Galaxy release statement on James Gunn
extract the movie title from the post:Guardians of the Galaxy
---
<input text here>
extract the movie title from the post:


Thus, we have connected to 2 different endpoints of the Co:here API; embed + generate.

## 2. DeepAI
The API we will try to use will be the Text to Image API. <br>
This API creates an image from scratch from a text description

In [30]:
# importing libraries

# Example posting a text URL:

import requests
r = requests.post(
    "https://api.deepai.org/api/text2img",
    data={
        'text': 'Example: posting a text URL',
    },
    headers={'api-key': '7f2f6ce1-98c4-404d-9e6f-ce0a8e65ea72'}
)
print(r.json())


# Example posting a local text file:

# import requests
# r = requests.post(
#     "https://api.deepai.org/api/text2img",
#     files={
#         'text': open('/path/to/your/file.txt', 'rb'),
#     },
#     headers={'api-key': '7f2f6ce1-98c4-404d-9e6f-ce0a8e65ea72'}
# )
# print(r.json())


# Example directly sending a text string:

import requests
r = requests.post(
    "https://api.deepai.org/api/text2img",
    data={
        'text': 'Example: directly sending a text string',
    },
    headers={'api-key': '7f2f6ce1-98c4-404d-9e6f-ce0a8e65ea72'}
)
print(r.json())

{'status': 'Out of free credits - please enter payment info in your dashboard: https://deepai.org/dashboard'}
{'status': 'Out of free credits - please enter payment info in your dashboard: https://deepai.org/dashboard'}


Requires payment information for the api to work. Therefore we will skip this API for now.

## 3. Muse by LightOn

Also requires payment information so we will forgo it for now

## 4. Bloom
This is a a large language model that is trained to continue and complete text from a prompt i.e. generate text in multiple languages (46)<br>
We'll be using hugging face Inference API which easily integrates NLP, audio and computer vision models deployed for inference.<br><br>
We'll try to use the Text Classification task under natural language processing. 

In [ ]:
#importing relevant libraries
from transformers import AutoModel

